<a href="https://colab.research.google.com/github/Divyaanshvats/CIFAR-with-CNN/blob/main/Model_CNN_EFFICIENTNETV0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Using EfficientNetV0 as a pre trained model**

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras import layers, models

(ds_train, ds_val), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
    with_info=True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.AM6D3V_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.


In [2]:
IMG_SIZE = 224
BATCH_SIZE = 32

In [3]:
def format_image(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = preprocess_input(image)
    return image, label

In [5]:
ds_train = ds_train.map(format_image).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
ds_val = ds_val.map(format_image).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [4]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False  # Freeze base model for transfer learning
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')  # Binary classification: cat vs dog
])

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,050,852 (15.45 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

# **Training Our Model**

In [7]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
history = model.fit(ds_train, validation_data=ds_val, epochs=5)


Epoch 1/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 86s 102ms/step - accuracy: 0.9483 - loss: 0.1592 - val_accuracy: 0.9903 - val_loss: 0.0309
Epoch 2/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 47s 65ms/step - accuracy: 0.9904 - loss: 0.0313 - val_accuracy: 0.9923 - val_loss: 0.0253
Epoch 3/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 38s 60ms/step - accuracy: 0.9914 - loss: 0.0262 - val_accuracy: 0.9923 - val_loss: 0.0234
Epoch 4/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 37s 63ms/step - accuracy: 0.9916 - loss: 0.0249 - val_accuracy: 0.9927 - val_loss: 0.0227
Epoch 5/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 36s 62ms/step - accuracy: 0.9930 - loss: 0.0220 - val_accuracy: 0.9929 - val_loss: 0.0226


# **Validating our Model**

In [8]:
loss, acc = model.evaluate(ds_val)
print(f"Validation Accuracy: {acc:.4f}")

146/146 ━━━━━━━━━━━━━━━━━━━━ 8s 51ms/step - accuracy: 0.9933 - loss: 0.0194
Validation Accuracy: 0.9929
